# Yelp API - Lab


## Introduction 

Now that we've seen how the Yelp API works and some basic Folium visualizations, it's time to put those skills to work in order to create a working map! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to: 
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and perform data analysis on the data returned
* Perform pagination to retrieve troves of data!
* Create a simple geographical system on to view information about selected businesses, at a given location. 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner. Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retrieving 50 at a time. Processes such as these are often referred to as pagination.

In this lab, you will define a search and then paginate over the results to retrieve all of the results. You'll then parse these responses as a DataFrame (for further exploration) and create a map using Folium to visualize the results geographically.

## Part I - Make the Initial Request

Start by making an initial request to the Yelp API. Your search must include at least 2 parameters: **term** and **location**. For example, you might search for pizza restaurants in NYC. The term and location is up to you, but make the request below.

In [2]:
#Your code here
import requests as r
client_id = "hKRSMf2mVCvsMxi0BymFCA"
api_key = "fJrWabTe3lM23x87ka3GiKEpNd80NJdmwdZL957_UWseItcNNcPzdWwl4mAq_jwfdmeSHGTNMhFTD6M3YzUOtINBPrl_C6CAU9eWPeRTF-8yGVikO5rDn0u0qSvcXHYx"

In [6]:
header = {"authorization": f"bearer {api_key}"}
params = {"location":"NYC","limit":50,"offset":50}
resp = r.get(
    "https://api.yelp.com/v3/businesses/search",
    params=params,
    headers=header)

In [16]:
def get_dict(offset):
    header = {"authorization": f"bearer {api_key}"}
    params = {"location":"NYC","limit":50,"offset":offset}
    resp = r.get(
        "https://api.yelp.com/v3/businesses/search",
        params=params,
        headers=header)
    return dict(resp.json())
def get_business_pages(amount):
    businesses = []
    for i in range(amount):
        businesses += get_dict(i*50)["businesses"]
    return businesses


In [8]:
r_dict = dict(resp.json())

In [9]:
r_dict.keys()

dict_keys(['businesses', 'total', 'region'])

In [10]:
r_dict["total"]

24900

In [11]:
r_dict["businesses"].__len__()

20

In [12]:
r_dict["businesses"][0]

{'id': 'H4jJ7XB3CetIr1pg56CczQ',
 'alias': 'levain-bakery-new-york',
 'name': 'Levain Bakery',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/jCdrXah--NjPaOLb-30BUw/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/levain-bakery-new-york?adjust_creative=hKRSMf2mVCvsMxi0BymFCA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=hKRSMf2mVCvsMxi0BymFCA',
 'review_count': 7515,
 'categories': [{'alias': 'bakeries', 'title': 'Bakeries'}],
 'rating': 4.5,
 'coordinates': {'latitude': 40.7799404643263, 'longitude': -73.980282552649},
 'transactions': [],
 'price': '$$',
 'location': {'address1': '167 W 74th St',
  'address2': '',
  'address3': '',
  'city': 'New York',
  'zip_code': '10023',
  'country': 'US',
  'state': 'NY',
  'display_address': ['167 W 74th St', 'New York, NY 10023']},
 'phone': '+12128746080',
 'display_phone': '(212) 874-6080',
 'distance': 8367.197123060287}

In [17]:
data = get_business_pages(5)

In [18]:
import pandas as pd
pd.Datadata

[{'id': 'H4jJ7XB3CetIr1pg56CczQ',
  'alias': 'levain-bakery-new-york',
  'name': 'Levain Bakery',
  'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/jCdrXah--NjPaOLb-30BUw/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/levain-bakery-new-york?adjust_creative=hKRSMf2mVCvsMxi0BymFCA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=hKRSMf2mVCvsMxi0BymFCA',
  'review_count': 7515,
  'categories': [{'alias': 'bakeries', 'title': 'Bakeries'}],
  'rating': 4.5,
  'coordinates': {'latitude': 40.7799404643263, 'longitude': -73.980282552649},
  'transactions': [],
  'price': '$$',
  'location': {'address1': '167 W 74th St',
   'address2': '',
   'address3': '',
   'city': 'New York',
   'zip_code': '10023',
   'country': 'US',
   'state': 'NY',
   'display_address': ['167 W 74th St', 'New York, NY 10023']},
  'phone': '+12128746080',
  'display_phone': '(212) 874-6080',
  'distance': 8367.197123060287},
 {'id': 'xEnNFXtMLDF5kZDxfaCJgA',
  'alias': 'the-

## Pagination

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. Your final goal will be to reformat the data concerning the businesses themselves into a pandas DataFrame from the json objects.

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and are also can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

In [ ]:
# Your code here; use a function or loop to retrieve all the results from your original request

## Exploratory Analysis

Take the restaurants from the previous question and do an initial exploratory analysis. At minimum, this should include looking at the distribution of features such as price, rating and number of reviews as well as the relations between these dimensions.

In [ ]:
#Your code here

## Mapping

Look at the initial Yelp example and try and make a map using Folium of the restaurants you retrieved. Be sure to also add popups to the markers giving some basic information such as name, rating and price.

In [ ]:
#Your code here

## Summary

Nice work! In this lab, you synthesized your skills for the day, making multiple API calls to Yelp in order to paginate through a results set, performing some basic exploratory analysis and then creating a nice map visual to display the results! Well done!